In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('default')
sns.set_palette("husl")

In [3]:
# df = pd.read_csv('your_dataset.csv')


# =====================================================
# PART 1: Feature Selection and Initial Analysis
# =====================================================

In [ ]:
clustering_features = [
    'transformable_price',  # Price (transformed for comparison)
    'building_size',        # Building area
    'rooms_count',          # Number of rooms
    'has_elevator',         # Elevator (comfort feature)
    'has_parking',          # Parking (essential amenity)
    'has_balcony',          # Balcony (lifestyle feature)
    'construction_year',    # Age of building
    'location_latitude',    # Geographic location
    'location_longitude'    # Geographic location
]

# Create a clean dataset for clustering
df_clustering = df[clustering_features].copy()

# Handle missing values
print("Missing values before cleaning:")
print(df_clustering.isnull().sum())

# =====================================================
# PART 2: Convert Geographic Coordinates to UTM
# =====================================================

In [ ]:
# Convert lat/lon to UTM coordinates
# Iran is primarily in UTM Zone 39N and 40N, we'll use 39N for most of the country
transformer = pyproj.Transformer.from_crs('EPSG:4326', 'EPSG:32639', always_xy=True)

utm_coords = []
for idx, row in df_clustering.iterrows():
    try:
        x, y = transformer.transform(row['location_longitude'], row['location_latitude'])
        utm_coords.append([x, y])
    except:
        utm_coords.append([np.nan, np.nan])

utm_coords = np.array(utm_coords)
df_clustering['utm_x'] = utm_coords[:, 0]
df_clustering['utm_y'] = utm_coords[:, 1]

In [ ]:
# Remove rows with invalid UTM coordinates
df_clustering = df_clustering.dropna(subset=['utm_x', 'utm_y'])

print(f"Dataset shape after UTM conversion: {df_clustering.shape}")


# =====================================================
# PART 3: K-Means with 10 Clusters (Initial Analysis)
# =====================================================


In [ ]:

# Prepare features for clustering (excluding original lat/lon)
features_for_kmeans = df_clustering.drop(['location_latitude', 'location_longitude'], axis=1)

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features_for_kmeans)

# Apply K-means with 10 clusters
kmeans_10 = KMeans(n_clusters=10, random_state=42, n_init=10)
clusters_10 = kmeans_10.fit_predict(X_scaled)

# Add cluster labels to dataframe
df_clustering['cluster_10'] = clusters_10

In [ ]:
# Create scatter plot for price vs UTM coordinates
plt.figure(figsize=(15, 10))

# Create subplot layout
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Plot 1: UTM coordinates colored by cluster
colors = plt.cm.tab10(np.linspace(0, 1, 10))
for i in range(10):
    cluster_data = df_clustering[df_clustering['cluster_10'] == i]
    ax1.scatter(cluster_data['utm_x'], cluster_data['utm_y'], 
               c=[colors[i]], label=f'Cluster {i}', alpha=0.6, s=30)

# Plot cluster centers in UTM space
utm_x_idx = list(features_for_kmeans.columns).index('utm_x')
utm_y_idx = list(features_for_kmeans.columns).index('utm_y')
centers_utm_x = scaler.inverse_transform(kmeans_10.cluster_centers_)[:, utm_x_idx]
centers_utm_y = scaler.inverse_transform(kmeans_10.cluster_centers_)[:, utm_y_idx]

ax1.scatter(centers_utm_x, centers_utm_y, c='red', marker='x', s=200, linewidths=3, label='Centroids')
ax1.set_xlabel('UTM X (meters)')
ax1.set_ylabel('UTM Y (meters)')
ax1.set_title('K-Means Clustering (k=10) - Geographic Distribution')
ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax1.grid(True, alpha=0.3)

In [ ]:
# Plot 2: Price vs UTM X coordinate colored by cluster
price_idx = list(features_for_kmeans.columns).index('transformable_price')
centers_price = scaler.inverse_transform(kmeans_10.cluster_centers_)[:, price_idx]

for i in range(10):
    cluster_data = df_clustering[df_clustering['cluster_10'] == i]
    ax2.scatter(cluster_data['utm_x'], cluster_data['transformable_price'], 
               c=[colors[i]], label=f'Cluster {i}', alpha=0.6, s=30)

ax2.scatter(centers_utm_x, centers_price, c='red', marker='x', s=200, linewidths=3, label='Centroids')
ax2.set_xlabel('UTM X (meters)')
ax2.set_ylabel('Transformable Price')
ax2.set_title('K-Means Clustering (k=10) - Price vs Location')
ax2.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# =====================================================
# PART 4: Optimal K Selection with Multiple Methods
# =====================================================


In [ ]:
# Test k values from 1 to 20
k_range = range(1, 21)
wcss = []  # Within-cluster sum of squares
silhouette_scores = []
distortions = []

In [ ]:
print("Computing metrics for different k values...")
for k in k_range:
    if k == 1:
        wcss.append(np.sum((X_scaled - X_scaled.mean(axis=0)) ** 2))
        silhouette_scores.append(0)
        distortions.append(np.sum((X_scaled - X_scaled.mean(axis=0)) ** 2))
    else:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(X_scaled)
        wcss.append(kmeans.inertia_)
        distortions.append(kmeans.inertia_)
        silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))
    
    print(f"k={k} completed")

In [ ]:
# Plot the results
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

# Elbow Method
ax1.plot(k_range, wcss, 'bo-')
ax1.set_xlabel('Number of clusters (k)')
ax1.set_ylabel('Within-Cluster Sum of Squares (WCSS)')
ax1.set_title('Elbow Method for Optimal k')
ax1.grid(True)

# Calculate the rate of change to find elbow
wcss_diff = np.diff(wcss)
wcss_diff2 = np.diff(wcss_diff)
ax2.plot(range(2, len(wcss_diff2)+2), wcss_diff2, 'ro-')
ax2.set_xlabel('Number of clusters (k)')
ax2.set_ylabel('Second Derivative of WCSS')
ax2.set_title('Second Derivative Method')
ax2.grid(True)

# Silhouette Analysis
ax3.plot(k_range[1:], silhouette_scores[1:], 'go-')  # Skip k=1
ax3.set_xlabel('Number of clusters (k)')
ax3.set_ylabel('Silhouette Score')
ax3.set_title('Silhouette Analysis')
ax3.grid(True)

In [ ]:
# Gap Statistic (simplified version)
# Generate random data with same dimensions for comparison
np.random.seed(42)
random_data = np.random.uniform(X_scaled.min(), X_scaled.max(), X_scaled.shape)
wcss_random = []

for k in k_range:
    if k == 1:
        wcss_random.append(np.sum((random_data - random_data.mean(axis=0)) ** 2))
    else:
        kmeans_random = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans_random.fit(random_data)
        wcss_random.append(kmeans_random.inertia_)

gap_stats = np.log(wcss_random) - np.log(wcss)
ax4.plot(k_range, gap_stats, 'mo-')
ax4.set_xlabel('Number of clusters (k)')
ax4.set_ylabel('Gap Statistic')
ax4.set_title('Gap Statistic Method')
ax4.grid(True)

plt.tight_layout()
plt.show()

In [ ]:

# Find optimal k using different methods
optimal_k_silhouette = k_range[1:][np.argmax(silhouette_scores[1:])] + 1  # +1 because we skip k=1
optimal_k_gap = k_range[np.argmax(gap_stats)]

In [ ]:

# Elbow method - find the point with maximum curvature
if len(wcss_diff2) > 0:
    optimal_k_elbow = np.argmax(wcss_diff2) + 2  # +2 because of double diff
else:
    optimal_k_elbow = 3

In [ ]:

print(f"\nOptimal k suggestions:")
print(f"Silhouette method: k = {optimal_k_silhouette}")
print(f"Gap statistic method: k = {optimal_k_gap}")
print(f"Elbow method (curvature): k = {optimal_k_elbow}")


In [ ]:

# Select the most frequent suggestion or use domain knowledge
suggested_ks = [optimal_k_silhouette, optimal_k_gap, optimal_k_elbow]
optimal_k = max(set(suggested_ks), key=suggested_ks.count)
print(f"\nSelected optimal k: {optimal_k}")

# =====================================================
# PART 5: DBSCAN Clustering
# =====================================================


In [ ]:
#Use only UTM coordinates and transformable price for DBSCAN
dbscan_features = df_clustering[['utm_x', 'utm_y', 'transformable_price']].copy()

# Standardize features for DBSCAN
scaler_dbscan = StandardScaler()
X_dbscan_scaled = scaler_dbscan.fit_transform(dbscan_features)


In [ ]:

# Grid search for optimal DBSCAN parameters to get 3 meaningful clusters
eps_values = np.arange(0.1, 2.0, 0.1)
min_samples_values = range(5, 50, 5)

best_score = -1
best_params = {'eps': 0.5, 'min_samples': 10}
best_n_clusters = 0

In [ ]:
print("Grid search for optimal DBSCAN parameters...")
print("Looking for parameters that produce exactly 3 clusters...")

for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(X_dbscan_scaled)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise = list(labels).count(-1)
        
        # We want exactly 3 clusters with reasonable noise level
        if n_clusters == 3 and n_noise < len(labels) * 0.3:  # Less than 30% noise
            try:
                score = silhouette_score(X_dbscan_scaled, labels)
                if score > best_score:
                    best_score = score
                    best_params = {'eps': eps, 'min_samples': min_samples}
                    best_n_clusters = n_clusters
                    print(f"Better params found: eps={eps:.1f}, min_samples={min_samples}, "
                          f"clusters={n_clusters}, noise={n_noise}, silhouette={score:.3f}")
            except:
                continue

print(f"\nBest DBSCAN parameters: eps={best_params['eps']}, min_samples={best_params['min_samples']}")
print(f"Best silhouette score: {best_score:.3f}")


In [ ]:

# Apply DBSCAN with best parameters
dbscan_final = DBSCAN(eps=best_params['eps'], min_samples=best_params['min_samples'])
dbscan_labels = dbscan_final.fit_predict(X_dbscan_scaled)

# Add DBSCAN labels to dataframe
df_clustering['dbscan_cluster'] = dbscan_labels


In [ ]:
#Count clusters and noise points
n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)

print(f"Number of clusters: {n_clusters_dbscan}")
print(f"Number of noise points: {n_noise}")

In [ ]:
# Visualize DBSCAN results
plt.figure(figsize=(20, 8))

# Plot 1: UTM coordinates
ax1 = plt.subplot(1, 2, 1)
unique_labels = set(dbscan_labels)
colors = plt.cm.Spectral(np.linspace(0, 1, len(unique_labels)))

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Noise points in black
        col = [0, 0, 0, 1]
        marker = 'x'
        label = 'Noise'
        size = 20
    else:
        marker = 'o'
        label = f'Cluster {k}'
        size = 30

    class_member_mask = (dbscan_labels == k)
    xy = X_dbscan_scaled[class_member_mask]
    
    # Convert back to original scale for plotting
    xy_original = scaler_dbscan.inverse_transform(xy)
    
    plt.scatter(xy_original[:, 0], xy_original[:, 1], 
               c=[col], marker=marker, s=size, alpha=0.6, label=label)

plt.xlabel('UTM X (meters)')
plt.ylabel('UTM Y (meters)')
plt.title(f'DBSCAN Clustering (eps={best_params["eps"]}, min_samples={best_params["min_samples"]})')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Price vs UTM X
ax2 = plt.subplot(1, 2, 2)
for k, col in zip(unique_labels, colors):
    if k == -1:
        col = [0, 0, 0, 1]
        marker = 'x'
        label = 'Noise'
        size = 20
    else:
        marker = 'o'
        label = f'Cluster {k}'
        size = 30

    class_member_mask = (dbscan_labels == k)
    cluster_data = df_clustering[class_member_mask]
    
    plt.scatter(cluster_data['utm_x'], cluster_data['transformable_price'], 
               c=[col], marker=marker, s=size, alpha=0.6, label=label)

plt.xlabel('UTM X (meters)')
plt.ylabel('Transformable Price')
plt.title('DBSCAN Clustering - Price vs Location')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# =====================================================
# HYPERPARAMETER ANALYSIS FOR DBSCAN
# =====================================================


In [ ]:

print("\n=== DBSCAN HYPERPARAMETER ANALYSIS ===")

print("DBSCAN Hyperparameter Effects:")
print("1. EPS (epsilon):")
print("   - Controls the maximum distance between two samples to be considered neighbors")
print("   - Smaller eps: More clusters, more noise points")
print("   - Larger eps: Fewer clusters, points merge into larger clusters")
print(f"   - Selected eps: {best_params['eps']}")

print("\n2. MIN_SAMPLES:")
print("   - Minimum number of samples in a neighborhood for a point to be considered core point")
print("   - Smaller min_samples: More core points, potentially more small clusters")
print("   - Larger min_samples: Fewer core points, more noise, denser clusters required")
print(f"   - Selected min_samples: {best_params['min_samples']}")

print(f"\nWith our selected parameters (eps={best_params['eps']}, min_samples={best_params['min_samples']}):")
print(f"- We achieved {n_clusters_dbscan} clusters as requested")
print(f"- Noise ratio: {n_noise/len(dbscan_labels)*100:.1f}%")
print(f"- Silhouette score: {best_score:.3f}")

In [ ]:

# Summary statistics for each cluster
print("\n=== CLUSTER SUMMARY STATISTICS ===")

# K-means clusters summary
print("K-means Clusters (k=10) Summary:")
for i in range(10):
    cluster_data = df_clustering[df_clustering['cluster_10'] == i]
    print(f"Cluster {i}: {len(cluster_data)} properties")
    print(f"  - Avg Price: {cluster_data['transformable_price'].mean():,.0f}")
    print(f"  - Avg Building Size: {cluster_data['building_size'].mean():.0f} m²")
    print(f"  - Avg Rooms: {cluster_data['rooms_count'].mean():.1f}")

print(f"\nDBSCAN Clusters Summary:")
for label in set(dbscan_labels):
    if label != -1:  # Skip noise
        cluster_data = df_clustering[df_clustering['dbscan_cluster'] == label]
        print(f"Cluster {label}: {len(cluster_data)} properties")
        print(f"  - Avg Price: {cluster_data['transformable_price'].mean():,.0f}")
        print(f"  - Avg Building Size: {cluster_data['building_size'].mean():.0f} m²")
        print(f"  - Geographic spread: UTM X range {cluster_data['utm_x'].max()-cluster_data['utm_x'].min():,.0f}m")

if n_noise > 0:
    noise_data = df_clustering[df_clustering['dbscan_cluster'] == -1]
    print(f"Noise points: {len(noise_data)} properties")
    print(f"  - These are outlier properties that don't fit well into any cluster")


In [ ]:

print("\n=== ANALYSIS COMPLETE ===")
print("The clustering analysis has identified meaningful property segments based on:")
print("- Geographic location (UTM coordinates)")
print("- Property characteristics (size, rooms, amenities)")
print("- Price levels")
print("This segmentation can be used for recommendation systems to suggest similar properties to users.")